## Data ingestion to Vector db

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

#### Read pdf's

In [3]:
from pathlib import Path
from typing import List
from langchain_community.document_loaders import PyMuPDFLoader


def process_pdfs(pdf_path):
    """
    Process all PDFs in a directory recursively using PyMuPDFLoader.

    """
    document_list = []
    pdf_dir = Path(pdf_path)

    if not pdf_dir.exists():
        raise FileNotFoundError(f"PDF directory not found: {pdf_dir}")

    files = list(pdf_dir.glob("**/*.pdf"))
    print(f"There are {len(files)} PDF files")

    for file in files:
        print(f"Processing file: {file.name}")

        try:
            loader = PyMuPDFLoader(str(file))
            docs = loader.load()

            for doc in docs:
                doc.metadata["source_file"] = file.name
                doc.metadata["source_path"] = str(file)
                doc.metadata["file_type"] = "pdf"
                doc.metadata["loader"] = "PyMuPDFLoader"

            document_list.extend(docs)
            print(f"Loaded {len(docs)} pages")

        except Exception as e:
            print(f"Error processing {file.name}: {e}")

    return document_list

pdf = process_pdfs("../data")

There are 3 PDF files
Processing file: Attention.pdf
Loaded 8 pages
Processing file: Supervised.pdf
Loaded 1 pages
Processing file: Large_language_model.pdf
Loaded 42 pages


In [5]:
pdf

[Document(metadata={'producer': 'Skia/PDF m143', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/143.0.0.0 Safari/537.36', 'creationdate': '2026-01-13T13:32:44+00:00', 'source': '../data/PDF_files/Attention.pdf', 'file_path': '../data/PDF_files/Attention.pdf', 'total_pages': 8, 'format': 'PDF 1.4', 'title': 'Attention (machine learning) - Wikipedia', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2026-01-13T13:32:44+00:00', 'trapped': '', 'modDate': "D:20260113133244+00'00'", 'creationDate': "D:20260113133244+00'00'", 'page': 0, 'source_file': 'Attention.pdf', 'source_path': '../data/PDF_files/Attention.pdf', 'file_type': 'pdf', 'loader': 'PyMuPDFLoader'}, page_content='Attention mechanism, overview\nAttention (machine learning)\nIn machine learning, attention is a method that determines the importance of each component in a sequence relative\nto the other components in that sequence. In natural language processing, importance

In [6]:

def document_split(documents, chunk_size= 500, chunk_overlap = 100):
    """
    Split documents into overlapping text chunks.

    Args:
        documents: LangChain documents
        size: Chunk size (characters)
        chunk_overlap: Overlap between chunks

    Returns:
        Chunked documents
    """

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""],
    )

    split_docs = splitter.split_documents(documents)

    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    if split_docs:
        print(f"Sample content: {split_docs[0].page_content[:200]}...")
        print(f"Sample metadata: {split_docs[0].metadata}")

    return split_docs






In [7]:
documents = document_split(pdf)

Split 51 documents into 457 chunks
Sample content: Attention mechanism, overview
Attention (machine learning)
In machine learning, attention is a method that determines the importance of each component in a sequence relative
to the other components in...
Sample metadata: {'producer': 'Skia/PDF m143', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/143.0.0.0 Safari/537.36', 'creationdate': '2026-01-13T13:32:44+00:00', 'source': '../data/PDF_files/Attention.pdf', 'file_path': '../data/PDF_files/Attention.pdf', 'total_pages': 8, 'format': 'PDF 1.4', 'title': 'Attention (machine learning) - Wikipedia', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2026-01-13T13:32:44+00:00', 'trapped': '', 'modDate': "D:20260113133244+00'00'", 'creationDate': "D:20260113133244+00'00'", 'page': 0, 'source_file': 'Attention.pdf', 'source_path': '../data/PDF_files/Attention.pdf', 'file_type': 'pdf', 'loader': 'PyMuPDFLoader'}


### Vector database and embedding

In [8]:
import chromadb
from chromadb.config import Settings
import uuid
import numpy as np 
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Any, Tuple, Dict

In [9]:


class EmbeddingHandler:
    def __init__(self, model_name = "all-MiniLm-L6-v2"):
        ##Here we initialise the embedding handler
        ## Args: model_name is an imported HuggingFace LLM for sentence embedding
        self.name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        # function for loading the SentenceTransformer
        try:
            print(f"Loading model {self.name}")
            self.model = SentenceTransformer(self.name)
            print(f"Model {self.name} loaded successfully!")
            print(f"Embedding dimensions {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error {e} while loading model {self.name}")

    

    def embedding_generation(self, text):
        # Creates the embeddings for the given texts
        # Embeddings are numpy array with shape ((Len(text), embedding_dimension)

        if not self.model:
            raise ValueError("Model is not loaded!")


        print(f"EMbedding generation for {len(text)} texts")
        
        embeddings = self.model.encode(text, show_progress_bar= True)
        print("Embeddings created successfully!")
        print(f"Embeddings shape: {embeddings.shape}")
        return embeddings

embedding_handler = EmbeddingHandler()
embedding_handler

Loading model all-MiniLm-L6-v2
Model all-MiniLm-L6-v2 loaded successfully!
Embedding dimensions 384


## Vectorstore

In [48]:

class VectorStore:
    def __init__(self, collection_name = "pdf_documents", persist_directory = "../data/VectorStore"):
        self.collection_name = collection_name
        self.dir = persist_directory
        self._initialize_store()

    
    def _initialize_store(self):
        try: 
            os.makedirs(self.dir,exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.dir)

            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata= {"description": "RAG's PDF embeddings"}
            )

            print(f"VectorStore initialized successfully!")
            print(f"Documents in the collection {self.collection_name}: {self.collection.count()}")
        
        except Exception as e:
            print(f"Error while initializing VectorStore {e}")
        
    
    def add_documents(self, documents, embeddings):

        if len(documents) != len(embeddings):
            raise ValueError("Embeddings and documents counts don't align")

        ids, metadatas, texts, vectors = [], [], [], []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            ids.append(f"doc_{uuid.uuid4().hex[:8]}_{i}")

            metadata = dict(doc.metadata)
            metadata["document_index"] = i
            metadata["content_length"] = len(doc.page_content)
            metadatas.append(metadata)

            texts.append(doc.page_content)
            vectors.append(embedding.tolist())

        self.collection.add(
            ids=ids,
            embeddings=vectors,
            metadatas=metadatas,
            documents=texts,
        )

        print(f"Added {len(documents)} documents to VectorStore")
        print(f"Total documents: {self.collection.count()}")

        
vectorStore = VectorStore()

vectorStore



VectorStore initialized successfully!
Documents in the collection pdf_documents: 457


In [49]:
content = [doc.page_content for doc in documents]

Embedding = embedding_handler.embedding_generation(content)

vectorStore.add_documents(documents,Embedding)


EMbedding generation for 457 texts


Batches: 100%|██████████| 15/15 [00:04<00:00,  3.35it/s]


Embeddings created successfully!
Embeddings shape: (457, 384)
Added 457 documents to VectorStore
Total documents: 914


## Rag retrieval pipeline

In [51]:
class RAG_retrieval:
    def __init__(self,vectorStore,embedding_handler):
        self.vectorStore = vectorStore
        self.embedding_handler = embedding_handler


    def retrieve(self, query, k = 5, threshold = 0):

        # Retrieves the top documents from the VectorStore for the given query

        embeddings_query = self.embedding_handler.embedding_generation([query])[0]

        try:
            results = self.vectorStore.collection.query(
                embeddings_queries = [embeddings_query.tolist()],
                n_results=k
            )

            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distance = results['distance'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids,documents,metadatas,distance)):

                    similarity_score = 1 - distance

                    if similarity_score >= threshold:
                        retrieved_docs.append({
                            'id':doc_id, 
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                    
                        print(f"Retrieved {len(retrieved_docs)} documents")

                    else:
                        print("Zero documents found")

        except Exception as e:
            print(f"Retrieval error {e}")
        
        return retrieved_docs


retriever = RAG_retrieval(vectorStore,embedding_handler)
        




In [52]:
class RAGRetrieval:
    def __init__(self, vector_store, embedding_handler, metric="cosine"):
        self.vector_store = vector_store
        self.embedding_handler = embedding_handler
        self.metric = metric
    

    def retrieve(self, query, k = 5, threshold= 0.0):
        """
        Retrieve top-k documents for a query.

        """
        print(f"retieve docs for query: {query}")
        
        query_embedding = self.embedding_handler.embedding_generation([query])[0]
        try:

            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=k,
            )

            retrieved_docs = []

            if not results.get("documents") or not results["documents"][0]:
                return []

            documents = results["documents"][0]
            metadatas = results["metadatas"][0]
            distances = results["distances"][0]
            ids = results["ids"][0]

            for rank, (doc_id, content, metadata, distance) in enumerate(
                zip(ids, documents, metadatas, distances), start=1
            ):
                if self.metric == "cosine":
                    similarity_score = 1 - distance
                else:
                    similarity_score = None  # leave raw distance only

                if similarity_score is None or similarity_score >= threshold:
                    retrieved_docs.append({
                        "id": doc_id,
                        "content": content,
                        "metadata": metadata,
                        "distance": distance,
                        "similarity_score": similarity_score,
                        "rank": rank,
                    })

            return retrieved_docs

        except Exception as e:
            raise RuntimeError(f"Retrieval failed: {e}")

retriever = RAGRetrieval(vectorStore,embedding_handler)

In [53]:
docs = retriever.retrieve("What is attention mechanism?")
print(docs[1]['content'])


retieve docs for query: What is attention mechanism?
EMbedding generation for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.64it/s]

Embeddings created successfully!
Embeddings shape: (1, 384)
34. Soydaner, Derya (August 2022). "Attention mechanism in neural
networks: where it comes and where it goes" (https://link.springer.c
om/10.1007/s00521-022-07366-3). Neural Computing and
Applications. 34 (16): 13371–13385. arXiv:2204.13154 (https://arxi
v.org/abs/2204.13154). doi:10.1007/s00521-022-07366-3 (https://do
i.org/10.1007%2Fs00521-022-07366-3). ISSN 0941-0643 (https://s
earch.worldcat.org/issn/0941-0643).
35. Britz, Denny; Goldie, Anna; Luong, Minh-Thanh; Le, Quoc (2017-


In [54]:
def build_context(retrieved_docs, max_chars=3000):
    context = ""

    for doc in retrieved_docs:
        source = doc["metadata"].get("source_file", "unknown")
        chunk = doc["content"]

        entry = f"\n[Source: {source}]\n{chunk}\n"

        if len(context) + len(entry) > max_chars:
            break

        context += entry

    return context.strip()


In [59]:
class RAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm

    def run(self, query, k=5):
        docs = self.retriever.retrieve(query, k=k)

        if not docs:
            return "No relevant documents found."

        context = build_context(docs)
        return self.llm.generate(query, context)


### OPENAI LLM 

In [56]:
from openai import OpenAI


class OpenAILLM:
    def __init__(self, model="gpt-4o-mini"):
        self.client = OpenAI(api_key=os.getenv("rag_key"))
        self.model = model

    def generate(self, query, context):
        prompt = f"""
                    Answer the question using ONLY the context below.
                    If the answer is not contained, say "I don't know".

                    Context:
                    {context}

                    Question:
                    {query}
                """

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a factual assistant grounded in retrieved documents."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1
        )

        return response.choices[0].message.content



In [58]:
llm = OpenAILLM()
rag = RAGPipeline(retriever, llm)

print(rag.run("What is the attention mechanism?"))


retieve docs for query: What is the attention mechanism?
EMbedding generation for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.15it/s]


Embeddings created successfully!
Embeddings shape: (1, 384)
The attention mechanism is a method in machine learning that determines the importance of each component in a sequence relative to the other components in that sequence. In natural language processing, this importance is represented by "soft" weights assigned to each word in a sentence. More generally, attention encodes vectors called token embeddings across a fixed-width sequence that can range from tens to millions of tokens in size.
